# Exploring Tools in LangChain

## Install OpenAI, and LangChain dependencies

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
# !pip install langchain==0.3.14
# !pip install langchain-openai==0.3.0
# !pip install langchain-community==0.3.14

## Install Data Extraction APIs

In [ ]:
# # to create custom tools
# !pip install wikipedia==1.4.0
# !pip install markitdown
# # to highlight json
# !pip install rich

## Enter Open AI API Key

In [ ]:
# from getpass import getpass

# OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [ ]:
# TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

## Enter WeatherAPI API Key

Get a free API key from [here](https://www.weatherapi.com/signup.aspx)

In [ ]:
# WEATHER_API_KEY = getpass('Enter WeatherAPI API Key: ')

## Setup Environment Variables

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
# import os

# os.environ['OPENAI_API_KEY'] = OPENAI_KEY
# os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## Exploring Built-in Tools

### Exploring the Wikipedia Tool

Enables you to tap into the Wikipedia API to search wikipedia pages for information

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=3,
                                       doc_content_chars_max=8000)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper, features="lxml")

In [ ]:
wiki_tool.description

In [ ]:
wiki_tool.args

In [ ]:
print(wiki_tool.invoke({"query": "Microsoft"}))

In [ ]:
print(wiki_tool.invoke({"query": "AI"}))

 You can customize the default tool with its own name, description and so on as follows

In [ ]:
from langchain.agents import Tool

wiki_tool_init = Tool(name="Wikipedia",
                      func=wiki_api_wrapper.run,
                      description="useful when you need a detailed answer about general knowledge")

In [ ]:
wiki_tool_init.description

In [ ]:
wiki_tool_init.args

In [ ]:
print(wiki_tool_init.invoke({"tool_input": "AI"}))

### Exploring the Tavily Search Tool

Tavily Search API is a search engine optimized for LLMs and RAG, aimed at efficient, quick and persistent search results

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5,
                                search_depth='advanced',
                                include_raw_content=True)

In [ ]:
tavily_tool.args

In [ ]:
tavily_tool.description

In [ ]:
results = tavily_tool.invoke("Tell me about Microsoft")
results

## Build your own tools in LangChain

Tools are interfaces that an agent, chain, or LLM can use to interact with the world. They combine a few things:

- The name of the tool
- A description of what the tool is
- JSON schema of what the inputs to the tool are
- The function to call
- Whether the result of a tool should be returned directly to the user

It is useful to have all this information because this information can be used to build action-taking systems! The name, description, and JSON schema can be used to prompt the LLM so it knows how to specify what action to take, and then the function to call is equivalent to taking that action.

### Building a Simple Math Tool

We will start by building a simple tool which does some basic math

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(a, b):
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

In [ ]:
type(multiply)

In [ ]:
multiply.invoke({"a": 2, "b": 3})

In [ ]:
multiply.invoke({"a": 2.1, "b": 3.2})

In [ ]:
multiply.invoke({"a": 2, "b": 'abc'})

Let's now build a tool with data type enforcing

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

class CalculatorInput(BaseModel):
    a: float = Field(description="first number")
    b: float = Field(description="second number")


def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

# we could also use the @tool decorator from before
multiply = StructuredTool.from_function(
    func=multiply,
    name="multiply",
    description="use to multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True
    )

# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

In [ ]:
multiply.invoke({"a": 2, "b": 3})

In [ ]:
# this code will error out as abc is not a floating point number
try:
    multiply.invoke({"a": 2, "b": 'abc'})
except Exception as e:
    print(e)

### Build a Web Search & Information Extraction Tool

In [ ]:
tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_raw_content=True)

result = tavily_tool.invoke("Tell me about Microsoft's Q4 2024 earning call report")
result

In [ ]:
result[0]['url']

In [ ]:
from markitdown import MarkItDown

md = MarkItDown()
doc_content = md.convert(result[0]['url'])
print(doc_content.title.strip())
print(doc_content.text_content)

In [ ]:
doc_content = md.convert(result[3]['url'])
print(doc_content.title.strip())
print(doc_content.text_content)

In [ ]:
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
import requests

tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_answer=False,
                                  include_raw_content=True)
md = MarkItDown()

@tool
def search_web_extract_info(query: str) -> list:
    """Search the web for a query and extracts useful information from the search links"""
    results = tavily_tool.invoke(query)
    docs = []
    for result in tqdm(results):
        # Extracting all text content from the URL
        try:
            extracted_info = md.convert(result['url'])
            text_title = extracted_info.title.strip()
            text_content = extracted_info.text_content.strip()
            docs.append(text_title + '\n' + text_content)
        except:
            print('Extraction blocked for url: ', result['url'])
            pass

    return docs

In [ ]:
docs = search_web_extract_info('OpenAI GPT-4o')

In [ ]:
from IPython.display import display, Markdown

display(Markdown(docs[0]))

### Build a Weather Tool

In [ ]:
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')

In [ ]:
import requests

@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather."""
    url = f"https://api.openweathermap.org/data/2.5/weather?q={query},IN&appid={WEATHER_API_KEY}&units=metric"

    response = requests.get(url)
    data = response.json()
    if data.get("name"):
        return data
    else:
        return "Weather Data Not Found"

In [ ]:
import rich

result = get_weather.invoke("Bangalore")
rich.print_json(data=result)

In [ ]:
import rich

result = get_weather.invoke("Kolkata")
rich.print_json(data=result)

## Explore LLM tool calling with custom tools

An agent is basically an LLM which has the capability to automatically call relevant functions to perform complex or tool-based tasks based on input human prompts.

Tool calling also popularly known as function calling is the ability to reliably enable such LLMs to call external tools and APIs.

We will leverate the custom tools we created earlier in the previous section and try to see if the LLM can automatically call the right tools based on input prompts

### Tool calling for LLMs with native support for tool or function calling

Tool calling allows a model to respond to a given prompt by generating output that matches a user-defined schema. While the name implies that the model is performing some action, this is actually not the case! The model is coming up with the arguments to a tool, and actually running the tool (or not) is up to the user or agent defined by the user.

Many LLM providers, including Anthropic, Cohere, Google, Mistral, OpenAI, and others, support variants of a tool calling feature. These features typically allow requests to the LLM to include available tools and their schemas, and for responses to include calls to these tools.



In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
tools = [multiply, search_web_extract_info, get_weather]
chatgpt_with_tools = chatgpt.bind_tools(tools)

In [ ]:
# LLMs are still not perfect in tool calling so you might need to play around with the following prompt
from langchain_core.messages import HumanMessage, ToolMessage

# prompt = """
#             Given only the tools at your disposal, mention tool calls for the following tasks:
#             Do not change the query given for any search tasks
#             1. What is 2.1 times 3.5
#             2. What is the current weather in Bangalore today
#             3. What are the 4 major Agentic AI Design Patterns
#          """

prompt = [
    HumanMessage(
        """
            Given only the tools at your disposal, mention tool calls for the following tasks:
            Do not change the query given for any search tasks
            1. What is 2.1 times 3.5
         """
    )
]

results = chatgpt_with_tools.invoke(prompt)

In [ ]:
prompt.append(results)
prompt

In [ ]:
results.tool_calls

In [ ]:
toolkit = {
    "multiply": multiply,
    "search_web_extract_info": search_web_extract_info,
    "get_weather": get_weather
}

for tool_call in results.tool_calls:
    selected_tool = toolkit[tool_call["name"].lower()]
    print(f"Calling tool: {tool_call['name']}")
    tool_output = selected_tool.invoke(tool_call["args"])
    print(tool_output)
    updated_prompt = prompt[:-1].append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
    print(updated_prompt)
    response = chatgpt_with_tools.invoke(updated_prompt)
    response.content

### Tool calling for LLMs without native support for tool or function calling

Some models like ChatGPT have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling.

Here we will explore an alternative method to invoke tools if you're using a model that does not natively support tool calling (even though we use ChatGPT here which supports it, we will assume it could be any LLM which doesn't support tool calling).

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

In [ ]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user instructions, for each instruction do the following:
 - Return the name and input of the tool to use.
 - Return your response as a JSON blob with 'name' and 'arguments' keys.
 - The `arguments` should be a dictionary, with keys corresponding
   to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}")
    ]
)

In [ ]:
instructions = [
                  {"input" : "What is 2.1 times 3.5"},
                  {"input" : "What is the current weather in Greenland"},
                  {"input" : "Tell me about the current state of Agentic AI in the industry" }
               ]

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

chain = (prompt
            |
         chatgpt
            |
         JsonOutputParser())

In [ ]:
responses = chain.map().invoke(instructions)

In [ ]:
responses

In [ ]:
toolkit = {
    "multiply": multiply,
    "search_web_extract_info": search_web_extract_info,
    "get_weather": get_weather
}

for tool_call in responses:
    selected_tool = toolkit[tool_call["name"].lower()]
    print(f"Calling tool: {tool_call['name']}")
    tool_output = selected_tool.invoke(tool_call["arguments"])
    print(tool_output)
    print()

In [ ]:
for doc in tool_output:
    print(doc)
    print()